## Preparing the workspace for CNTK in jupyter

In [1]:
#r "netstandard"
#load "Paket.fsx"

### 1. Install neccessary packages

In [ ]:
Paket.Dependencies.Install """
generate_load_scripts: true
storage: none
source https://nuget./api/v2
nuget CNTK.CPUOnly
nuget FsLab
nuget FSharp.Charting
"""

Usually in F# notebooks we tend to see <span style="color:green">paket.dependencies</span> created this way:

> Paket.Package ["CNTK.CPUOnly"; "FsLab"; "FSharp.Charting"]

I used <span style='color:#B21543'>Paket.Dependencies.Install</span> instead of <span style='color:#B21543'>Paket.Package</span> as it allows us to set **storage: none** and save some disk space and bandwidth by not re-downloading the same dependencies every time a project needs CNTK support, the required DLLs being rather immense as it is.

This is slightly ironic, considering that in order to use CNTK in jupyter we end up having to copy all dependencies in a common bin path anyway.

### 2. Reference Cntk.Core.Managed-#.#.dll from the nuget packages folder

In [ ]:
#load "Paket.Generated.Refs.fsx"

### 3. Copy all CNTK dependencies to the same folder

Currently all CNTK dlls need to be copied in the same folder with _Cntk.Core.Managed-#.#.dll_ else they can't be resolved at runtime. The following script finds the path of the presently referenced Cntk DLL and uses it to locate any dependencies. Should work for both GPU and CPUOnly.

Visual Studio does this by default in some projects, but here we need to do it by hand:

In [2]:
#load "prepareWorkspace.fsx"

let localBinFolder = "bin"

PrepareWorkspace.createOrCleanLocalBinFolder localBinFolder
PrepareWorkspace.copyDependenciesToLocalFolder localBinFolder (PrepareWorkspace.Release) 

Removing file 'bin\Cntk.Composite-2.6d.dll'...
Removing file 'bin\Cntk.Core-2.6d.dll'...
Removing file 'bin\Cntk.Core.CSBinding-2.6d.dll'...
Removing file 'bin\Cntk.Core.Managed-2.6.dll'...
Removing file 'bin\Cntk.Deserializers.Binary-2.6d.dll'...
Removing file 'bin\Cntk.Deserializers.HTK-2.6d.dll'...
Removing file 'bin\Cntk.Deserializers.Image-2.6d.dll'...
Removing file 'bin\Cntk.Deserializers.TextFormat-2.6d.dll'...
Removing file 'bin\Cntk.Math-2.6d.dll'...
Removing file 'bin\Cntk.PerformanceProfiler-2.6d.dll'...
Removing file 'bin\libiomp5md.dll'...
Removing file 'bin\mkldnn.dll'...
Removing file 'bin\mklml.dll'...
Removing file 'bin\opencv_world310d.dll'...
Removing file 'bin\zip.dll'...
Removing file 'bin\zlib.dll'...
Copying from 'packages\CNTK.CPUOnly\lib\netstandard2.0\Cntk.Core.Managed-2.6.dll'
Copying from 'packages\CNTK.CPUOnly\support\x64\Release\Cntk.Composite-2.6.dll'
Copying from 'packages\CNTK.CPUOnly\support\x64\Release\Cntk.Core-2.6.dll'
Copying from 'packages\CNTK.CP

So far I haven't been able to reference debug DLLs successfully in a notebook, i.e. without hitting the dreaded <span style='font-family:consolas'>DllNotFoundException: Cntk.Core.CSBinding-#.#.dll</span>, so I suggest you also stick to the release builds.

### 4. Reference CNTK from the newly created folder

<span style="color:blue">If you intend to use CNTK calls in this notebook you should restart the kernel after copying is complete, and then reference _Cntk.Core.Managed-#.#.dll_ from the newly created _/bin_ folder.</span>

In [2]:
#r @"bin\Cntk.Core.Managed-2.6.dll"
#load "Paket.Generated.Refs.fsx"

open CNTK
let cpu = DeviceDescriptor.UseDefaultDevice()
printfn "Congratulations, you are using CNTK for: %A" (cpu.Type)

Congratulations, you are using CNTK for: CPU


### 5. Update path variable

Finally, we need to update the path variable to inlcude the newly created /bin folder, otherwise we get an error when instantiating the learner before training.
>CNTK.CNTKLibPINVOKE.SGDLearner__SWIG_1 exception

<span style="color: blue">This is only affects the current process, so you'll have to do this again whenever you create a new notebook.</span>

In [ ]:
let path = System.Environment.GetEnvironmentVariable("PATH")
let path' = sprintf "%s%c%s" path (System.IO.Path.PathSeparator) (System.IO.Path.GetFullPath("bin"))
System.Environment.SetEnvironmentVariable("PATH", path')

 When calling CNTK from script make sure it's through a 64bit version of fsi.exe, such as fsiAnyCPU.exe, or you will run into _bad image_ exceptions.